logistische Regression

In [1]:
# Load the Pandas libraries, matplotlib and numpy
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

# Read data 
data = pd.read_csv("data/Umfragedaten_v1_an.csv") 

# Preview the first 5 lines of the loaded data 
data.head()

In [ ]:
#load libraries for regression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sn


Data often have NaN or other bad values, which means data need to be cleaned first.

In [ ]:
#need to dummy code variables first
x  = data['NETTO'].values.reshape(-1,1)
dummies = pd.get_dummies(data, columns=['RAUCH'])
y = dummies['RAUCH_JA'].values.reshape(-1,1)
dummies.head()

In [ ]:
#not sure why pandas creates 2D arrays, but need 1D otherwise logistic regression would crash
x=x[:,0]
y=y[:,0]

#need to clean data, there seem to be some NaN in x
print(np.any(np.isnan(x)))
print(np.any(np.isnan(y)))


In [ ]:
#print(np.where(np.isnan(x) == True))
print(x.ndim)
print(len(x))
index = np.where(~np.isnan(x))  #index of good data entries
#print(len(index))

xnew = x[index]  #only keep good entries
ynew = y[index]
print(xnew)

In [ ]:
#show a scatter plot
plt.plot(xnew,ynew,'o')
plt.xlabel('Einkommen')
plt.ylabel('Raucher (1=ja)')
plt.show()

#1=raucher, 0=nichtraucher

Now we train the model by splitting it into a training set and then the test set is used to validate it.

In [ ]:
#train the model

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(xnew.reshape(-1,1),ynew,test_size=0.05,random_state=0)


In [ ]:
# logistic regression
logreg = LogisticRegression(penalty=None)
#penalty=none heisst keine Regularisierung. Als Default benuetzt Python Regularisierung. 
#Was genau Regularisierung ist, wird man in mlr lernen.

# fit the model with data
logreg.fit(X_train,y_train)

#
y_pred=logreg.predict(X_test)
y_pred2 = logreg.predict_proba(X_test)[:,1]

print(logreg.coef_, logreg.intercept_)



In [ ]:
#confusion matrix
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

#the matrix shows that 37 data points were predicted wrongly, so this fit was not good.


In [ ]:
#to be sure which value are which
tn, fp, fn, tp = cnf_matrix.ravel()
print(tn,fp,fn,tp)
#so we have no true positives, but only true nagatives and false negatives.

In [ ]:
#plotting
#show a scatter plot
plt.plot(xnew,ynew,'o')
plt.plot(X_test,y_pred2)
plt.xlabel('Einkommen')
plt.ylabel('Raucher (1=ja)')
plt.show()

#this also shows a bad fit.

In [ ]:
#let's try changing the model by taking the (natural) log of the income
plt.plot(np.log(xnew),ynew,'o')
plt.xlabel('log(Einkommen)')
plt.ylabel('Raucher (1=ja)')
plt.show()
#one can see that there is actually not a very clear separation between non-smokers (0) and smokers (1). 
#We fit it anyway.

In [ ]:
xnew2 = np.log(xnew)
X_train,X_test,y_train,y_test = train_test_split(xnew2.reshape(-1,1),ynew,test_size=0.25, random_state=0)

# logistic regression 
logreg = LogisticRegression(penalty=None)

# fit the model with data
logreg.fit(X_train,y_train)

#
y_pred=logreg.predict(X_test)
y_pred2 = logreg.predict_proba(X_test)[:,1]

print(logreg.coef_, logreg.intercept_)



In [ ]:
#create the model manually to verify the prediction of python works (yes, it's the same)
pi = np.exp(-2.24+.19*xnew2) / (1+np.exp(-2.24+.19*xnew2))

#plotting
#show a scatter plot
plt.plot(xnew2,ynew,'o')
plt.plot(X_test,y_pred2)
plt.plot(xnew2,pi,color='b')
plt.xlabel('log(Einkommen)')
plt.ylabel('Raucher (1=ja)')
plt.show()


In [ ]:
#plot a larger x-axis to see the full model
xlarge = np.linspace(-15,45,60).reshape((-1,1))
ylarge = logreg.predict_proba(xlarge)[:,1]

plt.plot(xnew2,ynew,'o')
plt.plot(xlarge,ylarge,color='b')
plt.xlabel('log(Einkommen)')
plt.ylabel('Raucher (1=ja)')
plt.show()

#yes, the model is a logit model, but the data are not clearly separated, which means it doesn't fit too well.
# We also see that the model would always predict a nonsmoker within our data range.

In [ ]:
#wahrscheinlichkeit bei 2000 gehalt [%]
#predict() predicts a value, predict_proba predicts the probability
#I could not find a way to make it work with just 1 value, therefore I made
#an array and will ignore the result for 2001 below.
val = np.log(np.array([2000,2001]))

print(np.log(2000))
print((logreg.predict_proba(val.reshape(-1,1))*100.)[0])

#7.6 ist der x-Achsenwert  
#die zweite Zahl, also 31.1 ist die Wahrscheinlichkeit.


In [ ]:
print(logreg.score)